<a href="https://colab.research.google.com/github/Anu1996rag/Neural_Networks/blob/master/HyperParameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""THIS FILE DEMONSTRATES THE USE OF HYPERPARAMETER TUNING IN A NEURAL NETWORK """


# this file is to understand the basic regression problem using MLP

%tensorflow_version 2.x

!pip install scikit-learn==0.21.2 #this version I have taken so as to prevent from getting the below error 

""" 
RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f1c2c5e54d0>, 
as the constructor either does not set or modifies parameter learning_rate

"""

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

import numpy as np
np.seterr(invalid='ignore')

#loading the dataset

housing = fetch_california_housing()


#splitting the data into train and test first

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

#now , split the full training data into train and validation set

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full,y_train_full)

#data transformation (scaling the data before training it to the model

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

#build a model using function

def build_model(n_hidden=1,n_neurons = 30, learning_rate=3e-3,input_shape=X_train.shape[1:]):
    #defining layers
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons,activation='relu'))
    model.add(keras.layers.Dense(1))

    #compiling model
    model.compile(loss='mse',optimizer=keras.optimizers.SGD(lr=learning_rate))

    return model

#creating keras regressor based on the build model function
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

# #fitting the model
# keras_reg.fit(X_train,y_train,epochs=100,
#               validation_data=(X_valid,y_valid),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)])
#
# #model evaluation
# mse_test = keras_reg.score(X_test,y_test)
# print("Before using Randomized Search CV :",mse_test)

"""Using Randomized Search CV to search for the best hyperparameters """

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

#creating dictionary where we will specify the range of values
param_dict = {
                'n_hidden':[0,1,2],
                'n_neurons':[30,35,40],
                'learning_rate': reciprocal(3e-4,3e-2)
             }

#creating object
random_search_cv = RandomizedSearchCV(keras_reg,param_dict,cv=5)

#fitting the model
random_search_cv.fit(X_train,y_train,epochs=100,
              validation_data=(X_valid,y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

print("Best Parameters :",random_search_cv.best_params_)

print("Best Scores :",random_search_cv.best_score_)


Epoch 1/100
291/291 [==============================] - 1s 2ms/step - loss: 2.3483 - val_loss: 7.5038
Epoch 2/100
291/291 [==============================] - 0s 2ms/step - loss: 1.0549 - val_loss: 0.9322
Epoch 3/100
291/291 [==============================] - 0s 1ms/step - loss: 0.8023 - val_loss: 0.8811
Epoch 4/100
291/291 [==============================] - 0s 1ms/step - loss: 0.7369 - val_loss: 0.7778
Epoch 5/100
291/291 [==============================] - 0s 2ms/step - loss: 0.6964 - val_loss: 0.8040
Epoch 6/100
291/291 [==============================] - 0s 1ms/step - loss: 0.6682 - val_loss: 0.7600
Epoch 7/100
291/291 [==============================] - 0s 1ms/step - loss: 0.6450 - val_loss: 0.6836
Epoch 8/100
291/291 [==============================] - 0s 1ms/step - loss: 0.6235 - val_loss: 0.6423
Epoch 9/100
291/291 [==============================] - 0s 1ms/step - loss: 0.6044 - val_loss: 0.5924
Epoch 10/100
291/291 [==============================] - 0s 1ms/step - loss: 0.5872 - val_lo

In [6]:
random_search_cv.best_estimator_.get_params()

{'build_fn': <function __main__.build_model>,
 'learning_rate': 0.00900548655211988,
 'n_hidden': 2,
 'n_neurons': 35}

In [7]:
random_search_cv.score(X_test,y_test)

162/162 [==============================] - 0s 862us/step - loss: 0.2881


-0.2881025969982147